<u>**Name: Haridharan M / 
Work Location : Pune  /
Hometown : Coimbatore  / 
Link : [Portfolio](https://haridharan05.github.io/MyCreativeSpace/)**</u>

In [7]:
# Azure Blob Storage
spark.conf.set("fs.azure.account.key.intdpcollplfstr02.blob.core.windows.net", 
               "xxxxxxxxxxxxxxxxxxxxxx=")

# Azure paths - full and incremental load
full_load_base = "wasbs://gs-dbo-db@intdpcollplfstr02.blob.core.windows.net/gs-lead-db/Onetime_full_load"
incremental_base = "wasbs://gs-dbo-db@intdpcollplfstr02.blob.core.windows.net/gs-lead-db/incremental_file_2025_04_05"

#Fabric path - Staging
staging_base = "abfss://Platform_Support@onelake.dfs.fabric.microsoft.com/GALAXYBRIDGE.Lakehouse/Tables"



StatementMeta(, 07164f15-156d-41a4-aaf4-51722abebe75, 9, Finished, Available, Finished)

In [8]:
# Dictionary
full_paths = {
    "fact_averagecosts": f"{full_load_base}/fact.averagecosts.dlm",
    "fact_transactions": f"{full_load_base}/fact.transactions.dlm",
    "hier_clnd": f"{full_load_base}/hier.clnd.dlm",
    "hier_hldy": f"{full_load_base}/hier.hldy.dlm",
    "hier_invloc": f"{full_load_base}/hier.invloc.dlm",
    "hier_invstatus": f"{full_load_base}/hier.invstatus.dlm",
    "hier_possite": f"{full_load_base}/hier.possite.dlm",
    "hier_pricestate": f"{full_load_base}/hier.pricestate.dlm",
    "hier_prod": f"{full_load_base}/hier.prod.dlm",
    "hier_rtlloc": f"{full_load_base}/hier.rtlloc.dlm"
}

incremental_paths = {
    "fact_averagecosts": f"{incremental_base}/fact.averagecosts_2025_04_05.dlm",
    "fact_transactions": f"{incremental_base}/fact.transactions_2025_04_05.dlm",
    "hier_clnd": f"{incremental_base}/hier.clnd_2025_04_05.dlm",
    "hier_hldy": f"{incremental_base}/hier.hldy_2025_04_05.dlm",
    "hier_invloc": f"{incremental_base}/hier.invloc_2025_04_05.dlm",
    "hier_invstatus": f"{incremental_base}/hier.invstatus_2025_04_05.dlm",
    "hier_possite": f"{incremental_base}/hier.possite_2025_04_05.dlm",
    "hier_pricestate": f"{incremental_base}/hier.pricestate_2025_04_05.dlm",
    "hier_prod": f"{incremental_base}/hier.prod_2025_04_05.dlm",
    "hier_rtlloc": f"{incremental_base}/hier.rtlloc_2025_04_05.dlm"
}

StatementMeta(, 07164f15-156d-41a4-aaf4-51722abebe75, 10, Finished, Available, Finished)

In [9]:
# Schemas - Fact and DIM Tables

from pyspark.sql.types import StructType, StringType, DoubleType, DateType

# Schema for fact_transactions
transactions_schema = (StructType()
    .add("order_id", StringType(), True)            
    .add("line_id", StringType(), True)             
    .add("type", StringType(), True)
    .add("dt", StringType(), True)                  
    .add("pos_site_id", StringType(), True) 
    .add("sku_id", StringType(), True)   
    .add("fscldt_id", StringType(), True)
    .add("price_substate_id", StringType(), True)
    .add("sales_units", DoubleType(), True)
    .add("sales_dollars", DoubleType(), True)
    .add("discount_dollars", DoubleType(), True)
    .add("original_order_id", StringType(), True)
    .add("original_line_id", StringType(), True)
)

# Schema for fact_averagecosts
avgcosts_schema = (StructType()
    .add("fscldt_id", StringType(), True)
    .add("sku_id", StringType(), True)
    .add("average_unit_standardcost", DoubleType(), True)
    .add("average_unit_landedcost", DoubleType(), True)
)

# Schema for hier_clnd (Calendar)
clnd_schema = (StructType()
    .add("fscldt_id", StringType(), True)
    .add("fscldt_label", StringType(), True)
    .add("fsclwk_id", StringType(), True)
    .add("fsclwk_label", StringType(), True)
    .add("fsclmth_id", StringType(), True)
    .add("fsclmth_label", StringType(), True)
    .add("fsclqrtr_id", StringType(), True)
    .add("fsclqrtr_label", StringType(), True)
    .add("fsclyr_id", StringType(), True)
    .add("fsclyr_label", StringType(), True)
    .add("ssn_id", StringType(), True)
    .add("ssn_label", StringType(), True)
    .add("ly_fscldt_id", StringType(), True)
    .add("lly_fscldt_id", StringType(), True)
    .add("fscldow", StringType(), True)
    .add("fscldom", StringType(), True)
    .add("fscldoq", StringType(), True)
    .add("fscldoy", StringType(), True)
    .add("fsclwoy", StringType(), True)
    .add("fsclmoy", StringType(), True)
    .add("fsclqoy", StringType(), True)
    .add("date", DateType(), True)
)

# Schema for hier_possite
possite_schema = (StructType()
    .add("site_id", StringType(), True)
    .add("site_label", StringType(), True)
    .add("subchnl_id", StringType(), True)
    .add("subchnl_label", StringType(), True)
    .add("chnl_id", StringType(), True)
    .add("chnl_label", StringType(), True)
)

# Schema for hier_pricestate
pricestate_schema = (StructType()
    .add("substate_id", StringType(), True)
    .add("substate_label", StringType(), True)
    .add("state_id", StringType(), True)
    .add("state_label", StringType(), True)
)

# Schema for hier_prod
prod_schema = (StructType()
    .add("sku_id", StringType(), True)
    .add("sku_label", StringType(), True)
    .add("stylclr_id", StringType(), True)
    .add("stylclr_label", StringType(), True)
    .add("styl_id", StringType(), True)
    .add("styl_label", StringType(), True)
    .add("subcat_id", StringType(), True)
    .add("subcat_label", StringType(), True)
    .add("cat_id", StringType(), True)
    .add("cat_label", StringType(), True)
    .add("dept_id", StringType(), True)
    .add("dept_label", StringType(), True)
    .add("issvc", StringType(), True)
    .add("isasmbly", StringType(), True)
    .add("isnfs", StringType(), True)
)

# Schema for hier_rtlloc
rtlloc_schema = (StructType()
    .add("str", StringType(), True)
    .add("str_label", StringType(), True)
    .add("dstr", StringType(), True)
    .add("dstr_label", StringType(), True)
    .add("rgn", StringType(), True)
    .add("rgn_label", StringType(), True)
)

# Schema for hier_invstatus
invstatus_schema = (StructType()
    .add("code_id", StringType(), True)
    .add("code_label", StringType(), True)
    .add("bckt_id", StringType(), True)
    .add("bckt_label", StringType(), True)
    .add("ownrshp_id", StringType(), True)
    .add("ownrshp_label", StringType(), True)
)

# Schema for hier_hldy
hldy_schema = (StructType()
    .add("hldy_id", StringType(), True)
    .add("hldy_label", StringType(), True)
)

# Schema for hier_invloc
invloc_schema = (StructType()
    .add("loc", StringType(), True)
    .add("loc_label", StringType(), True)
    .add("loctype", StringType(), True)
    .add("loctype_label", StringType(), True)
)

StatementMeta(, 07164f15-156d-41a4-aaf4-51722abebe75, 11, Finished, Available, Finished)

In [10]:
# Process Full Load Data and Write to Staging Delta Tables in Lakehouse Folder
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StringType, DoubleType, DateType

# fact_transactions
fact_transactions_path = f"{staging_base}/fact_transactions"
df_full_transactions = spark.read \
    .option("header", "true") \
    .option("sep", "|") \
    .schema(transactions_schema) \
    .csv(full_paths["fact_transactions"])
df_full_transactions_clean = (df_full_transactions
    .dropna(subset=["order_id", "line_id", "pos_site_id", "sku_id", "fscldt_id"])
    .withColumn("dt", F.to_timestamp(F.col("dt"), "yyyy-MM-dd'T'HH:mm:ss"))
    .filter((F.col("sales_units") >= 0) &
            (F.col("sales_dollars") >= 0) &
            (F.col("discount_dollars") >= 0))
    .filter(F.col("dt") >= F.to_timestamp(F.lit("2015-01-01T00:00:00"), "yyyy-MM-dd'T'HH:mm:ss"))
)
df_full_transactions_clean.write.format("delta").mode("overwrite").save(fact_transactions_path)

# fact_averagecosts
fact_avgcosts_path = f"{staging_base}/fact_averagecosts"
df_full_avgcosts = spark.read \
    .option("header", "true") \
    .option("sep", "|") \
    .schema(avgcosts_schema) \
    .csv(full_paths["fact_averagecosts"])
df_full_avgcosts_clean = df_full_avgcosts.dropna(subset=["fscldt_id", "sku_id"])
df_full_avgcosts_clean.write.format("delta").mode("overwrite").save(fact_avgcosts_path)

# hier_clnd
dim_clnd_path = f"{staging_base}/dim_clnd"
df_full_clnd = spark.read \
    .option("header", "true") \
    .option("sep", "|") \
    .schema(clnd_schema) \
    .csv(full_paths["hier_clnd"])
df_full_clnd_clean = df_full_clnd.dropna(subset=["fscldt_id"]) \
    .withColumn("date", F.to_date(F.col("date"), "yyyy-MM-dd"))
df_full_clnd_clean.write.format("delta").mode("overwrite").save(dim_clnd_path)

# hier_possite
dim_possite_path = f"{staging_base}/dim_possite"
df_full_possite = spark.read \
    .option("header", "true") \
    .option("sep", "|") \
    .schema(possite_schema) \
    .csv(full_paths["hier_possite"])
df_full_possite_clean = df_full_possite.dropna(subset=["site_id"]) \
    .withColumn("site_id", F.trim(F.col("site_id")))
df_full_possite_clean.write.format("delta").mode("overwrite").save(dim_possite_path)

# hier_prod
dim_prod_path = f"{staging_base}/dim_prod"
df_full_prod = spark.read \
    .option("header", "true") \
    .option("sep", "|") \
    .schema(prod_schema) \
    .csv(full_paths["hier_prod"])
df_full_prod_clean = df_full_prod.dropna(subset=["sku_id"])
df_full_prod_clean.write.format("delta").mode("overwrite").save(dim_prod_path)

# hier_pricestate
dim_pricestate_path = f"{staging_base}/dim_pricestate"
df_full_pricestate = spark.read \
    .option("header", "true") \
    .option("sep", "|") \
    .schema(pricestate_schema) \
    .csv(full_paths["hier_pricestate"])
df_full_pricestate_clean = df_full_pricestate.dropna(subset=["substate_id"])
df_full_pricestate_clean.write.format("delta").mode("overwrite").save(dim_pricestate_path)

# hier_rtlloc
dim_rtlloc_path = f"{staging_base}/dim_rtlloc"
df_full_rtlloc = spark.read \
    .option("header", "true") \
    .option("sep", "|") \
    .schema(rtlloc_schema) \
    .csv(full_paths["hier_rtlloc"])
df_full_rtlloc_clean = df_full_rtlloc.dropna(subset=["str"])
df_full_rtlloc_clean.write.format("delta").mode("overwrite").save(dim_rtlloc_path)

# hier_invstatus
dim_invstatus_path = f"{staging_base}/dim_invstatus"
df_full_invstatus = spark.read \
    .option("header", "true") \
    .option("sep", "|") \
    .schema(invstatus_schema) \
    .csv(full_paths["hier_invstatus"])
df_full_invstatus_clean = df_full_invstatus.dropna(subset=["code_id"])
df_full_invstatus_clean.write.format("delta").mode("overwrite").save(dim_invstatus_path)

# hier_hldy
dim_hldy_path = f"{staging_base}/dim_hldy"
df_full_hldy = spark.read \
    .option("header", "true") \
    .option("sep", "|") \
    .schema(hldy_schema) \
    .csv(full_paths["hier_hldy"])
df_full_hldy_clean = df_full_hldy.dropna(subset=["hldy_id"])
df_full_hldy_clean.write.format("delta").mode("overwrite").save(dim_hldy_path)

# hier_invloc (dimension)
dim_invloc_path = f"{staging_base}/dim_invloc"
df_full_invloc = spark.read \
    .option("header", "true") \
    .option("sep", "|") \
    .schema(invloc_schema) \
    .csv(full_paths["hier_invloc"])
df_full_invloc_clean = df_full_invloc.dropna(subset=["loc"])
df_full_invloc_clean.write.format("delta").mode("overwrite").save(dim_invloc_path)

print("Full load data has been processed and written to staging (Lakehouse folder) successfully.")

StatementMeta(, 07164f15-156d-41a4-aaf4-51722abebe75, 12, Finished, Available, Finished)

Full load data has been processed and written to staging (Lakehouse folder) successfully.


In [11]:
from delta.tables import DeltaTable
import pyspark.sql.functions as F

# HELPER FUNCTION

def process_table(final_path, full_df, incr_df, merge_key_cols, update_exprs):
    """
    final_path: Destination Delta table.
    full_df:    df - full load (cleaned).
    incr_df:    df - incremental load (cleaned).
    merge_key_cols: List of column names used as keys for merge.
    update_exprs:   Dictionary mapping - to update expressions for whenMatchedUpdate().
    """
    # Deduplicate incremental data based on merge keys
    incr_df = incr_df.dropDuplicates(merge_key_cols)

    # merge - "t" for target and "s" for source
    condition = " AND ".join([f"t.{col} = s.{col}" for col in merge_key_cols])
    print("Merge Condition:", condition)

    # Check if the Delta table already exists
    if DeltaTable.isDeltaTable(spark, final_path):
        print(f"Delta table exists at {final_path}. Merging incremental data...")
        delta_table = DeltaTable.forPath(spark, final_path)
        delta_table.alias("t").merge(
            incr_df.alias("s"),
            condition
        ).whenMatchedUpdate(set=update_exprs) \
         .whenNotMatchedInsertAll() \
         .execute()
    else:
        # If table doesn't exist, create it by union of full + incremental
        print(f"Delta table does not exist at {final_path}. Creating table by union of full and incremental data.")
        df_final = full_df.unionByName(incr_df)
        df_final.write.format("delta").mode("overwrite").save(final_path)


# LOG DB FOLDER
log_db_path = "abfss://Platform_Support@onelake.dfs.fabric.microsoft.com/GALAXYBRIDGE.Lakehouse/Tables/"


# 1. fact_transactions

# Paths & Keys
log_fact_tx_path = log_db_path + "fact_transactions"
incr_fact_tx_path = incremental_paths["fact_transactions"]
fact_tx_keys = ["order_id", "line_id", "pos_site_id", "sku_id", "fscldt_id", "price_substate_id", "type"]
fact_tx_update = {
    "sales_units":      "t.sales_units + s.sales_units",
    "sales_dollars":    "t.sales_dollars + s.sales_dollars",
    "discount_dollars": "t.discount_dollars + s.discount_dollars"
}

# Read & Clean Incremental Data
df_incr_transactions = spark.read \
    .option("header", "true") \
    .option("sep", "|") \
    .schema(transactions_schema) \
    .csv(incr_fact_tx_path)

df_incr_transactions_clean = (df_incr_transactions
    .dropna(subset=["order_id", "line_id", "pos_site_id", "sku_id", "fscldt_id"])
    .withColumn("dt", F.to_timestamp(F.col("dt"), "yyyy-MM-dd'T'HH:mm:ss"))
    .filter((F.col("sales_units") >= 0) & (F.col("sales_dollars") >= 0) & (F.col("discount_dollars") >= 0))
    .filter(F.col("dt") >= F.to_timestamp(F.lit("2015-01-01T00:00:00"), "yyyy-MM-dd'T'HH:mm:ss"))
)

# Merge
process_table(log_fact_tx_path, df_full_transactions_clean, df_incr_transactions_clean, fact_tx_keys, fact_tx_update)


# 2. fact_averagecosts
log_avgcosts_path = log_db_path + "fact_averagecosts"
incr_avgcosts_path = incremental_paths["fact_averagecosts"]
avgcosts_keys = ["fscldt_id", "sku_id"]
avgcosts_update = {
    "average_unit_standardcost": "s.average_unit_standardcost",
    "average_unit_landedcost":   "s.average_unit_landedcost"
}

df_incr_avgcosts = spark.read \
    .option("header", "true") \
    .option("sep", "|") \
    .schema(avgcosts_schema) \
    .csv(incr_avgcosts_path)

df_incr_avgcosts_clean = df_incr_avgcosts.dropna(subset=["fscldt_id", "sku_id"])

process_table(log_avgcosts_path, df_full_avgcosts_clean, df_incr_avgcosts_clean, avgcosts_keys, avgcosts_update)


# 3. hier_clnd
log_clnd_path = log_db_path + "dim_clnd"
incr_clnd_path = incremental_paths["hier_clnd"]
clnd_keys = ["fscldt_id"]

df_incr_clnd = spark.read \
    .option("header", "true") \
    .option("sep", "|") \
    .schema(clnd_schema) \
    .csv(incr_clnd_path)

df_incr_clnd_clean = df_incr_clnd.dropna(subset=["fscldt_id"]).withColumn("date", F.to_date(F.col("date"), "yyyy-MM-dd"))

process_table(log_clnd_path, df_full_clnd_clean, df_incr_clnd_clean, clnd_keys, {})

# 4. hier_possite

log_possite_path = log_db_path + "dim_possite"
incr_possite_path = incremental_paths["hier_possite"]
possite_keys = ["site_id"]

df_incr_possite = spark.read \
    .option("header", "true") \
    .option("sep", "|") \
    .schema(possite_schema) \
    .csv(incr_possite_path)

df_incr_possite_clean = df_incr_possite.dropna(subset=["site_id"]).withColumn("site_id", F.trim(F.col("site_id")))

process_table(log_possite_path, df_full_possite_clean, df_incr_possite_clean, possite_keys, {})

# 5. hier_prod 
log_prod_path = log_db_path + "dim_prod"
incr_prod_path = incremental_paths["hier_prod"]
prod_keys = ["sku_id"]

df_incr_prod = spark.read \
    .option("header", "true") \
    .option("sep", "|") \
    .schema(prod_schema) \
    .csv(incr_prod_path)

df_incr_prod_clean = df_incr_prod.dropna(subset=["sku_id"])

process_table(log_prod_path, df_full_prod_clean, df_incr_prod_clean, prod_keys, {})


# 6. hier_pricestate

log_pricestate_path = log_db_path + "dim_pricestate"
incr_pricestate_path = incremental_paths["hier_pricestate"]
pricestate_keys = ["substate_id"]

df_incr_pricestate = spark.read \
    .option("header", "true") \
    .option("sep", "|") \
    .schema(pricestate_schema) \
    .csv(incr_pricestate_path)

df_incr_pricestate_clean = df_incr_pricestate.dropna(subset=["substate_id"])

process_table(log_pricestate_path, df_full_pricestate_clean, df_incr_pricestate_clean, pricestate_keys, {})

# 7. hier_rtlloc

log_rtlloc_path = log_db_path + "dim_rtlloc"
incr_rtlloc_path = incremental_paths["hier_rtlloc"]
rtlloc_keys = ["str"]

df_incr_rtlloc = spark.read \
    .option("header", "true") \
    .option("sep", "|") \
    .schema(rtlloc_schema) \
    .csv(incr_rtlloc_path)

df_incr_rtlloc_clean = df_incr_rtlloc.dropna(subset=["str"])

process_table(log_rtlloc_path, df_full_rtlloc_clean, df_incr_rtlloc_clean, rtlloc_keys, {})


# 8. hier_invstatus

log_invstatus_path = log_db_path + "dim_invstatus"
incr_invstatus_path = incremental_paths["hier_invstatus"]
invstatus_keys = ["code_id"]

df_incr_invstatus = spark.read \
    .option("header", "true") \
    .option("sep", "|") \
    .schema(invstatus_schema) \
    .csv(incr_invstatus_path)

df_incr_invstatus_clean = df_incr_invstatus.dropna(subset=["code_id"])

process_table(log_invstatus_path, df_full_invstatus_clean, df_incr_invstatus_clean, invstatus_keys, {})


# 9. hier_hldy 

log_hldy_path = log_db_path + "dim_hldy"
incr_hldy_path = incremental_paths["hier_hldy"]
hldy_keys = ["hldy_id"]

df_incr_hldy = spark.read \
    .option("header", "true") \
    .option("sep", "|") \
    .schema(hldy_schema) \
    .csv(incr_hldy_path)

df_incr_hldy_clean = df_incr_hldy.dropna(subset=["hldy_id"])

process_table(log_hldy_path, df_full_hldy_clean, df_incr_hldy_clean, hldy_keys, {})


# 10. hier_invloc 

log_invloc_path = log_db_path + "dim_invloc"
incr_invloc_path = incremental_paths["hier_invloc"]
invloc_keys = ["loc"]

df_incr_invloc = spark.read \
    .option("header", "true") \
    .option("sep", "|") \
    .schema(invloc_schema) \
    .csv(incr_invloc_path)

df_incr_invloc_clean = df_incr_invloc.dropna(subset=["loc"])

process_table(log_invloc_path, df_full_invloc_clean, df_incr_invloc_clean, invloc_keys, {})

print("Incremental load merge complete. Final log DB tables are updated.")


StatementMeta(, 07164f15-156d-41a4-aaf4-51722abebe75, 13, Finished, Available, Finished)

Merge Condition: t.order_id = s.order_id AND t.line_id = s.line_id AND t.pos_site_id = s.pos_site_id AND t.sku_id = s.sku_id AND t.fscldt_id = s.fscldt_id AND t.price_substate_id = s.price_substate_id AND t.type = s.type
Delta table exists at abfss://Platform_Support@onelake.dfs.fabric.microsoft.com/GALAXYBRIDGE.Lakehouse/Tables/fact_transactions. Merging incremental data...
Merge Condition: t.fscldt_id = s.fscldt_id AND t.sku_id = s.sku_id
Delta table exists at abfss://Platform_Support@onelake.dfs.fabric.microsoft.com/GALAXYBRIDGE.Lakehouse/Tables/fact_averagecosts. Merging incremental data...
Merge Condition: t.fscldt_id = s.fscldt_id
Delta table exists at abfss://Platform_Support@onelake.dfs.fabric.microsoft.com/GALAXYBRIDGE.Lakehouse/Tables/dim_clnd. Merging incremental data...
Merge Condition: t.site_id = s.site_id
Delta table exists at abfss://Platform_Support@onelake.dfs.fabric.microsoft.com/GALAXYBRIDGE.Lakehouse/Tables/dim_possite. Merging incremental data...
Merge Condition: 

In [12]:
# Enriched Refined Aggregated Table for Reporting

log_db_path = "abfss://Platform_Support@onelake.dfs.fabric.microsoft.com/GALAXYBRIDGE.Lakehouse/Tables/"

# Define paths
log_fact_tx_path = log_db_path + "fact_transactions"
log_avgcosts_path = log_db_path + "fact_averagecosts"
log_clnd_path = log_db_path + "dim_clnd"
log_possite_path = log_db_path + "dim_possite"
log_prod_path = log_db_path + "dim_prod"
log_pricestate_path = log_db_path + "dim_pricestate"

# Read Delta tables - log DB folder
df_final_tx = spark.read.format("delta").load(log_fact_tx_path)
df_final_avg = spark.read.format("delta").load(log_avgcosts_path)
df_final_clnd = spark.read.format("delta").load(log_clnd_path)
df_final_possite = spark.read.format("delta").load(log_possite_path)
df_final_prod = spark.read.format("delta").load(log_prod_path)
df_final_pricestate = spark.read.format("delta").load(log_pricestate_path)

# Aggregate logic
df_enriched = df_final_tx.alias("ft") \
    .join(df_final_possite.alias("ps"), df_final_tx.pos_site_id == df_final_possite.site_id, "left") \
    .join(df_final_prod.alias("p"), "sku_id", "left") \
    .join(df_final_clnd.alias("clnd"), "fscldt_id", "left") \
    .join(df_final_pricestate.alias("psate"), df_final_tx.price_substate_id == df_final_pricestate.substate_id, "left") \
    .select(
        "ft.pos_site_id",
        "ps.site_label",
        "ft.sku_id",
        "p.sku_label",
        "ft.fscldt_id",
        "clnd.fscldt_label",
        "clnd.fsclwk_id",
        "clnd.fsclmth_label",
        "ft.price_substate_id",
        "psate.substate_label",
        "ft.type",
        "ft.sales_units",
        "ft.sales_dollars",
        "ft.discount_dollars",
        "ft.order_id",
        "ft.line_id"
    )

# Join with fact_averagecosts for cost details
df_enriched = df_enriched.alias("e") \
    .join(df_final_avg.alias("avg"), 
          (df_enriched.fscldt_id == df_final_avg.fscldt_id) & (df_enriched.sku_id == df_final_avg.sku_id), 
          "left") \
    .selectExpr("e.*", "avg.average_unit_standardcost", "avg.average_unit_landedcost")

# display(df_enriched.limit(10))

# final refined aggregated logic
df_refined = df_enriched.groupBy(
    "pos_site_id", "site_label", "sku_id", "sku_label",
    "fscldt_id", "fscldt_label", "fsclwk_id", "fsclmth_label",
    "price_substate_id", "substate_label", "type"
).agg(
    F.sum("sales_units").alias("total_sales_units"),
    F.sum("sales_dollars").alias("total_sales_dollars"),
    F.sum("discount_dollars").alias("total_discount_dollars"),
    F.avg("average_unit_standardcost").alias("avg_standard_cost"),
    F.avg("average_unit_landedcost").alias("avg_landed_cost")
)

# sample refined data
print("Sample of final refined aggregated data:")
display(df_refined.limit(10))

# output - log DB
mview_weekly_sales_enriched_path = log_db_path + "mview_weekly_sales_enriched"

# log DB folder (overwrite mode) with schema merging.
df_refined.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .option("mergeSchema", "true") \
    .save(mview_weekly_sales_enriched_path)

print("Enriched refined aggregated table has been written to:", mview_weekly_sales_enriched_path)
print("Full ETL pipeline execution completed successfully.")


StatementMeta(, 07164f15-156d-41a4-aaf4-51722abebe75, 14, Finished, Available, Finished)

Sample of enriched data:


SynapseWidget(Synapse.DataFrame, f6786414-c5a2-48fe-b96d-19d2d953ff38)

Sample of final refined aggregated data:


SynapseWidget(Synapse.DataFrame, d9681c55-3105-458a-96f1-30c500b87cfe)

Enriched refined aggregated table has been written to: abfss://Platform_Support@onelake.dfs.fabric.microsoft.com/GALAXYBRIDGE.Lakehouse/Tables/mview_weekly_sales_enriched
Full ETL pipeline execution completed successfully.


In [15]:
df = spark.sql("SELECT * FROM GALAXYBRIDGE.mview_weekly_sales_enriched")
display(df)

StatementMeta(, 07164f15-156d-41a4-aaf4-51722abebe75, 20, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ceffce54-71b8-4716-a652-da385213456e)

In [22]:
%%sql
SELECT 
    type, 
    price_substate_id, 
    COUNT(1) AS Total_trans_count,
    SUM(total_sales_dollars) AS total_sales_dollars, 
    SUM(total_discount_dollars) AS total_discount_dollars, 
    SUM(avg_standard_cost) AS total_standard_cost, 
    SUM(avg_landed_cost) AS total_landed_cost 
FROM GALAXYBRIDGE.mview_weekly_sales_enriched
GROUP BY type, price_substate_id
ORDER BY type ASC


StatementMeta(, 07164f15-156d-41a4-aaf4-51722abebe75, 27, Finished, Available, Finished)

<Spark SQL result set with 12 rows and 7 fields>